# Week 2

1. Description of the problem and discussion of the background.

I live in Paris for almost 10 years. Each year, I have many friends who come to visit Paris. When they ask me which activities to do or which places to go, I always recommend them to go visiting famous places like Eiffel Tower, Louvre Museum, Notre Dame Cathedral, ... and go to some restaurants that I used to go often.

Now with the help of this FourSquare platform, I might do some studies to find with evidences what are the best things to do around best places in Paris, find the best "one, two or three-day visit in Paris" that we can visit famous places in Paris with less time of travel.

2. Description of the data and how it will be used to solve the problem.

The data we use is from the page wikipedia: https://fr.wikipedia.org/wiki/Tourisme_en_France. It means Tourisme in France, it had statistics about number of visitors for top visited places in France where we can find top visited places in Paris. We then find a hotel with good prices at the center of the place that we can be closer to all of these places.

Then we will find the location of each places and do some clustering to define some group of places that are close to the others, so each time we visit each group so the time for travel is less then if we just visit randomly.

1. Import all the libraries that we need in this project

In [12]:
# install Folium
! pip install folium==0.5.0

In [13]:
import requests
from IPython.core.display import HTML
from IPython.display import Image
import folium  # plotting library
from pandas.io.json import json_normalize
# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
import requests  # library to handle requests
import pandas as pd  # library for data analsysis
import numpy as np  # library to handle data in a vectorized manner
import random  # library for random number generation

import pandas as pd
import numpy as np
import requests

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

2. List of top visited places in France in 2016 from Wikipedia: https://fr.wikipedia.org/wiki/Tourisme_en_France

In [14]:
url = "https://fr.wikipedia.org/wiki/Tourisme_en_France"

# Read a list of dataframe
tables = pd.read_html(url)
df = tables[2]
#df0 = df0.set_index("Postal Code")
df.head(10)

,Rang 2016,Site ou événement,Commune,2012,2013,2014,2015,2016
0,1,Disneyland,Marne-la-Vallée,16 000 000,14 900 000,14 200 000,14 800 000,13 400 000
1,2,Musée du Louvre,Paris,9 660 609,9 201 157,9 134 612,8 422 000,6 989 000
2,3,Domaine de Versailles,Versailles,7 278 745,7 527 121,7 700 000,7 357 903,6 701 193
3,4,Tour Eiffel,Paris,6 270 000,6 740 000,7 097 302,6 917 000,5 934 000
4,5,Centre Pompidou,Paris,3 800 000,3 745 000,3 450 000,3 060 000,3 335 509
5,6,Musée d'Orsay,Paris,3 579 130,3 467 320,3 480 609,3 439 832,2 997 622
6,7,Site du Mont-Saint-Michel,Le Mont-Saint-Michel,2 300 000,2 200 000,2 257 000,2 331 640,2 254 254
7,8,Cité des sciences et de l'industrie de La Vill...,Paris,2 641 356,2 642 255,2 676 000,2 013 046,2 196 194
8,9,Le Puy du Fou (Grand Parc + Cinéscénie),Les Epesses,1 600 000,1 740 000,1 912 000,2 050 000,2 150 000
9,10,Parc du Futuroscope,Chasseneuil-du-Poitou,1 730 000,1 460 000,1 650 000,1 830 000,1 900 000


3. List of top visited places in Paris

In [15]:
df0 = df[df["Commune"] == "Paris"]
df0 = df0[["Site ou événement", "Commune", "2016"]]
df0 = df0.rename(columns={"Site ou événement": "Place", "Commune": "City"})
df0.reset_index(drop=True, inplace=True)
df0.loc[18,"Place"] = "Panthéon, Paris"

df0

,Place,City,2016
0,Musée du Louvre,Paris,6 989 000
1,Tour Eiffel,Paris,5 934 000
2,Centre Pompidou,Paris,3 335 509
3,Musée d'Orsay,Paris,2 997 622
4,Cité des sciences et de l'industrie de La Vill...,Paris,2 196 194
5,Muséum national d'histoire naturelle,Paris,1 586 450
6,Arc de Triomphe,Paris,1 342 361
7,Musée de l'Armée,Paris,1 206 065
8,Musée du quai Branly,Paris,1 151 922
9,Musée de la Musique,Paris,1 139 674


In [16]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'KOMVDCBWZS0Y3AFXRQIXOEZHXUM5J4LGZSJF1VYMMWITCXTK'  # your Foursquare ID
# your Foursquare Secret
CLIENT_SECRET = 'G2NHNKYLVV3YKBT2UVQ34NYTLEQCO2G4WVI3ABXLYVZSOH35'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: KOMVDCBWZS0Y3AFXRQIXOEZHXUM5J4LGZSJF1VYMMWITCXTK
CLIENT_SECRET:G2NHNKYLVV3YKBT2UVQ34NYTLEQCO2G4WVI3ABXLYVZSOH35


4. We search for the center of all places where we are close to all places

In [17]:
latitude = []
longitude = []
i = 0
for adr in df0["Place"]:
    try:
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(adr)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        i = i + 1
    except:
        pass

zone_latitude = np.average(latitude)
zone_longitude = np.average(longitude)
print("Latitude: ", zone_latitude)
print("Longitude: ", zone_longitude)


Latitude:  48.861836296
Longitude:  2.328988868876115


5. We visualize the map with the center in red and all places in green

In [18]:
# generate map centred around the center of all visited places
center_map = folium.Map(location=[zone_latitude, zone_longitude], zoom_start=13)

# add a red circle marker to represent the center of all visited places
folium.CircleMarker(
    [zone_latitude, zone_longitude],
    radius=10,
    color='red',
    popup='Center of all visited places',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(center_map)


for i in range(len(latitude)):
    folium.CircleMarker(
        [latitude[i], longitude[i]],
        radius=10,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(center_map)


# display map
center_map

6. We look for the hotel around the center (about 500 m) so we can recommend some hotels to people visiting Paris.

In [19]:
# Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel.
search_query = 'Hotel'
radius = 500
print(search_query + ' .... OK!')

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, zone_latitude, zone_longitude, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results = requests.get(url).json()

# Get relevant part of JSON and transform it into a pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)


# Define information of interest and filter dataframe
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + \
    [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(
    get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split(
    '.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

Hotel .... OK!


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Hôtel Montalembert,Hotel,3 rue de Montalembert,Rue du Bac,48.856874,2.327702,"[{'label': 'display', 'lat': 48.8568738875692,...",560,75007,FR,Paris,Île-de-France,France,"[3 rue de Montalembert (Rue du Bac), 75007 Par...",4adcda04f964a520253221e3
1,Hôtel Normandy,Hotel,7 Rue de l'Échelle,NaN,48.864219,2.334387,"[{'label': 'display', 'lat': 48.86421850100031...",476,75001,FR,Paris,Île-de-France,France,"[7 Rue de l'Échelle, 75001 Paris, France]",4b73059ff964a52060962de3
2,Hôtel Saint James Albany,Hotel,202 rue de Rivoli,NaN,48.864295,2.330813,"[{'label': 'display', 'lat': 48.86429481803056...",304,75001,FR,Paris,Île-de-France,France,"[202 rue de Rivoli, 75001 Paris, France]",4bf23e5d52bda5939672b2b7
3,Hôtel Le Meurice,Hotel,228 rue de Rivoli,NaN,48.865333,2.328137,"[{'label': 'display', 'lat': 48.8653326, 'lng'...",394,75001,FR,Paris,Île-de-France,France,"[228 rue de Rivoli, 75001 Paris, France]",4b8b0cbef964a5203b9032e3
4,Hôtel Brighton,Hotel,218 Rue de Rivoli,NaN,48.864851,2.329005,"[{'label': 'display', 'lat': 48.86485102956639...",335,75001,FR,Paris,Île-de-France,France,"[218 Rue de Rivoli, 75001 Paris, France]",4adcda04f964a5200c3221e3
5,Hôtel Renaissance Paris Vendôme,Hotel,4 rue du Mont Thabor,NaN,48.865478,2.329598,"[{'label': 'display', 'lat': 48.8654778858979,...",407,75001,FR,Paris,Île-de-France,France,"[4 rue du Mont Thabor, 75001 Paris, France]",4adcda03f964a520c23121e3
6,Hôtel d'Orsay,Hotel,93 rue de Lille,NaN,48.860394,2.323172,"[{'label': 'display', 'lat': 48.860394394553, ...",455,75007,FR,Paris,Île-de-France,France,"[93 rue de Lille, 75007 Paris, France]",4b851a7af964a520264c31e3
7,Hôtel Du Louvre - In the Unbound Collection by...,Hotel,Place André Malraux,Place André Malraux,48.862983,2.335667,"[{'label': 'display', 'lat': 48.8629830120211,...",505,75001,FR,Paris,Île-de-France,France,"[Place André Malraux (Place André Malraux), 75...",4adcda04f964a520073221e3
8,Hôtel Emeraude Louvre Montana,Hotel,12 rue Saint-Roch,NaN,48.864438,2.331710,"[{'label': 'display', 'lat': 48.86443825084932...",351,75001,FR,Paris,Île-de-France,France,"[12 rue Saint-Roch, 75001 Paris, France]",4d8d2205d4ec8cfae31d5289
9,Hôtel du Quai Voltaire (L'),Hotel,19 Quai Voltaire,NaN,48.858886,2.331303,"[{'label': 'display', 'lat': 48.8588864507748,...",369,75007,FR,Paris,Île-de-France,France,"[19 Quai Voltaire, 75007 Paris, France]",4adcd9fff964a520813021e3


7. We visualize the red center and all hotels near that

In [20]:
# generate map centred around the center of all visited places
hotels_map = folium.Map(
    location=[zone_latitude, zone_longitude], zoom_start=15)

# add a red circle marker to represent the center of all visited places
folium.CircleMarker(
    [zone_latitude, zone_longitude],
    radius=10,
    color='red',
    popup='Center of all visited places',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(hotels_map)

# add the hotels as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)
    
hotels_map

8. We now plan for some trips around Paris. But there are so many places to go, so we should have some groups of places that we will visit at the same time, for exemple: 1 group per day. We use K means to find n groups, n = 7 in this case.

In [21]:
places = []
latitude_array = np.asarray(latitude)
longitude_array = np.asarray(longitude)
places = np.concatenate((latitude_array,longitude_array))
places = places.reshape(2,len(latitude)).transpose()

# 3 clusters, run n times
k_means = KMeans(init="k-means++", n_clusters=7, n_init=30)

k_means.fit(places)

# generate map centred around the center of all visited places
clusters_map = folium.Map(
    location=[zone_latitude, zone_longitude], zoom_start=13)

def f(x):
    return {
        0:'orange',
        1:'yellow',
        2:'green',
        3:'blue',
        4:"purple",
        5:"black",
        6:"white"
    }[x]

for i in range(len(latitude)):
    folium.CircleMarker(
        [latitude[i], longitude[i]],
        radius=10,
        color=f(k_means.labels_[i]),
        fill=True,
        fill_color=f(k_means.labels_[i]),
        fill_opacity=0.6
    ).add_to(clusters_map)

# display map
clusters_map
